For this demonstration we will be simulating a SrTiO3 CeO2 interface downloaded from https://www.materialscloud.org/explore/stoceriaitf/grid/calculations. We will need to do some manipulations to make the structure suitable for TEM simulation

In [ ]:
import pyms
import numpy as np
import torch

# for creating a responsive plot
%matplotlib widget
# Get crystal
crystal = pyms.structure.fromfile(
    "Structures/SrTiO3_CeO2_interface.xyz", atomic_coordinates="cartesian"
)

# A few maniupulations to remove vaccuum at edges and create a psuedo-periodic
# structure
crystal = crystal.resize([0.1, 0.76], axis=0)
from copy import deepcopy

other = deepcopy(crystal).resize([0.017, 0.99], axis=0)
other.reflect([0])
crystal = crystal.concatenate(other, axis=0)

# Output structure for examination in Vesta
# crystal.output_vesta_xtl("manipulated.xtl")

# Quick plot of crystal
crystal.quickplot(atomscale=0.1,block=False,)

Now lets run the calculation and plot the results

In [ ]:
# Probe accelerating voltage in eV
eV = 3e5

# Objective aperture in mrad, set to None for no aperture
app = 15

# Set up series of thicknesses
thicknesses = np.arange(100, 801, 200)

# Set up  series of defocii
df = np.arange(-400, 401, 200)

output,Aperpix = pyms.HRTEM(
    crystal,
    eV,
    app,
    thicknesses,
    df=df,
    showProgress='notebook'
)


# Crop output to focus on interface
output = output[...,:output.shape[-2]//2,:]

import matplotlib.pyplot as plt
ny,nx = output.shape[-2:]
# Plot images
fig, ax = plt.subplots(figsize=(16,16))
ax.set_xticks(np.arange(nx/2,(len(thicknesses)+0.5)*nx,nx))
ax.set_xticklabels(thicknesses)
ax.set_xlabel('Thickness ($\AA$)')
ax.set_yticks(np.arange(ny/2,(len(df)+0.5)*ny,ny))
ax.set_yticklabels(df)
ax.set_ylabel('Defocus ($\AA$)')


# Stack defocii (first dimension of output) and thicknesses (second dimension of output)
# to make a thickness defocus series
montage = np.hstack(np.hstack(output))
ax.imshow(montage)

# Plot a white border to differentiate images in the montage
xlim = ax.get_xlim()
ylim = ax.get_ylim()
for i in range(len(thicknesses)):
    ax.plot([i*output.shape[-1],i*output.shape[-1]],[0,montage.shape[-2]],'w-')

for i in range(len(df)):
    ax.plot([0,montage.shape[-1]],[i*output.shape[-2],i*output.shape[-2]],'w-')
ax.set_xlim(xlim)
ax.set_ylim(ylim)